In [48]:
import torch
import torch.nn as nn
from torch.nn.functional import relu, sigmoid, conv2d, dropout, max_pool2d, softmax
import static_token_div.tools.vocab_tools as vocab_tools

In [49]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Using gpu: %s ' % torch.cuda.is_available())

Using gpu: False 


In [50]:
vocab = vocab_tools.Vocab(emb_filename="embedding.txt")

In [51]:
X_train = vocab.matrice[:2524]
X_test = vocab.matrice[2525:]

(torch.Size([2524, 100]),)

In [53]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(100, 32)
        self.l2 = nn.Linear(32, vocab.vocab_size)
    def forward(self, x):
        x = relu(self.l1(x))
        x = softmax(self.l2(x))
        return x

In [58]:
def cross_entropy_loss(y):
    return -torch.log(y)

In [59]:
model = MLP()
optimizer = torch.optim.Adam(lr=1e-3, params=model.parameters())
batch_size = 32

In [60]:
def train(model, X_train, nb_epoch=20):
    history = {
        "train_loss": [],
        "test_loss": []
    }
    batch_start = torch.arange(0, len(X_train), batch_size)
    for epoch in range(nb_epoch):
        model.train()
        current_loss = 0
        for start in batch_start:
            X_batch = X_train[start:start+batch_size]
            output = model(X_batch)

            loss = cross_entropy_loss(output)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            current_loss += loss.item()
        avg_loss = current_loss / len(batch_start)
        history["train_loss"].append(avg_loss)

        print(f'Epoch [{epoch+1}/{nb_epoch}], Loss: {avg_loss:.4f}')


In [61]:
train(model, X_train)

RuntimeError: grad can be implicitly created only for scalar outputs